### Langchain e llamacpp
Questo notebook contiene un esempio di utilizzo di langchain invocando un modello "locale" grazie a llamacpp.

E' necessario lanciare il server OPENAI utilizzando le istruzioni presenti nel repo llamacpp.

Grazie a llamacpp è possibile simulare "in locale" un server compatibile con le API OpenAI (vedi http://localhost:8000/docs)

In [1]:
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )


In [3]:
output = llm.invoke("Spiegami la meccanica quantistica")
print(output.content)

 Spiegando la meccanica quantistica significa spiegare come la matematica e le osservazioni scientifiche si coniugano per descrivere il comportamento dei sistemi fisici a livello atomico o subatomico. La meccanica quantistica è una teoria che descrive i processi fisici che coinvolgono particelle a livello atomico o subatomico, e che differenza i processi a livello classico, in cui i movimenti sono determinati da leggi che descrivono i moti continuo e determinato.

La meccanica quantistica è stata sviluppata in tre età:

1. Età atomica (1857-1927): nel 1857, l'atomista russa Dmitri Mendeleev propose che tutti gli elementi si dividessero in unità indivisibili, ovvero atomi. Nello stesso anno, l'inglese William Thomson scoprì che l'elettricità si trasformava in calore, che portò alla teoria dell'elettricità e della calore. Nel 1900, l'inglese J.J. Thomson scoprì l'esistenza degli isotopi, ovvero atomi che hanno lo stesso numero di particelle cariche ma differenti masse. Nel 1913, l'ingles

In [4]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage)

messages =[
    SystemMessage(content="you are a fhisician and respond only in italian"),
    HumanMessage(content="Spiegami la meccanica quantistica")
]

output = llm.invoke(messages)
print(output.content)

 La meccanica quantistica è una branca della fisica che descrive le proprietà dei corpi materiali e energetici, e i loro rapporti, utilizzando principi e metodi matematici come l'analisi matematica.


### Streaming
Per gestire la risposta con lo streaminig

In [5]:
chunks = []
async for chunk in llm.astream("hello. tell me something about yourself"):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

 Hello! I'm an AI language model, designed to help people find information and answer their questions. What would you like to know?

### Caching

In memory cache

In [6]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )

In [7]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

prompt = 'Tell me a joke that a toddler can understand'
llm.invoke(prompt)

CPU times: user 168 ms, sys: 8.61 ms, total: 176 ms
Wall time: 3.39 s


". Why don't scientists trust atoms? Because they make up everything!"

In [8]:
%%time
llm.invoke(prompt)

CPU times: user 138 µs, sys: 61 µs, total: 199 µs
Wall time: 202 µs


". Why don't scientists trust atoms? Because they make up everything!"

In [9]:
%%time
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


prompt = 'Tell me a joke that a toddler can understand'
llm.invoke(prompt)

CPU times: user 37.9 ms, sys: 23.3 ms, total: 61.2 ms
Wall time: 60.8 ms


'. Why did the chicken cross the road? To get to the other side! Why did the tomato turn red? Because it saw the salad dressing!'

In [10]:
%%time
llm.invoke(prompt)

CPU times: user 1.25 ms, sys: 380 µs, total: 1.63 ms
Wall time: 1.73 ms


'. Why did the chicken cross the road? To get to the other side! Why did the tomato turn red? Because it saw the salad dressing!'

### Streaming

In [11]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1'
    )

In [12]:
prompt = "Racconta una storia sulla luna e le stelle"
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

 Once upon a time, in a galaxy far away, there was a little raccoon named Rocky. He lived on a small planet called Earth with his family and friends. Rocky was always fascinated by the night sky, especially the moon and the stars.

One night, Rocky's mother told him a story about the moon and the stars. She said that long ago, the moon was just a rock floating in space, but one day it was chosen to be the guardian of the night sky. The other stars were so jealous of the moon that they decided to challenge it to a race.

The challenge was simple: whoever could run around the planet faster would be the winner. The stars were confident that they could win, but the moon was determined to prove them wrong.

The race began, and the stars zoomed off into the night sky. But the moon was clever and used its gravity to pull itself closer to the earth, giving it an advantage. The stars started to slow down, but the moon kept going faster and faster until it crossed the finish line first.

The oth

### PromptTemplates

In [13]:
from langchain.prompts import PromptTemplate 
template = '''You are an expert vitologist. Write a few sentences about that virus {virus} in {language}'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='italian')

In [14]:
prompt

'You are an expert vitologist. Write a few sentences about that virus hiv in italian'

In [15]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        temperature=0
    
    )
output = llm.invoke(prompt)
print(output.content)

 Il virus HIV, o Human Immunodeficiency Virus, è un virus che attacca e distrugge le cellule dell'immunosistema umano, causando la malattia dell'AIDS. È trasmesso principalmente attraverso il contatto sessuale non protetto, ma anche attraverso l'inoculazione di sangue infettato o attraverso la nascita da madre a figlio. Il virus è in grado di mutare rapidamente, rendendo difficile la creazione di farmaci efficaci per il trattamento.


### ChatPromptTemplate

In [16]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content = 'You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} cities in  {country} by population order by population descending.')
    ]
)

messages = chat_template.format_messages(n = '10', country = 'Italy')
print(messages)

[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 10 cities in  Italy by population order by population descending.')]


In [42]:
llm = ChatOpenAI(
        api_key='545454545',
        base_url='http://localhost:8000/v1',
        temperature=0
    
    )
output = llm.invoke(messages)
print(output.content)

 {
"cities": [
{
"name": "Milan",
"population": 2693847
},
{
"name": "Rome",
"population": 2833020
},
{
"name": "Naples",
"population": 2187553
},
{
"name": "Florence",
"population": 1946847
},
{
"name": "Torino",
"population": 1904423
},
{
"name": "Venice",
"population": 1509189
},
{
"name": "Genoa",
"population": 1567187
},
{
"name": "Bologna",
"population": 1087244
},
{
"name": "Pisa",
"population": 97734
},
{
"name": "Reggio Emilia",
"population": 92657
}
]
}
